In [ ]:
from qiskit_dynamics import Solver, DynamicsBackend
from qiskit_dynamics.backend import default_experiment_result_function
from qiskit_dynamics.array import Array
import jax
from qiskit.providers.fake_provider import *
from qiskit import pulse

In [ ]:
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import JordanWignerMapper, ParityMapper
import numpy as np
from qiskit_nature.second_q.formats.molecule_info import MoleculeInfo
from qiskit_nature.second_q.drivers import PySCFDriver

gate_backend = FakeManila()
gate_backend.configuration().hamiltonian['qub'] = {'0': 2,'1': 2,'2': 2,'3': 2,'4': 2}
jax.config.update("jax_enable_x64", True)
jax.config.update("jax_platform_name", "cpu")
Array.set_default_backend("jax")
pulse_backend = DynamicsBackend.from_backend(gate_backend, evaluation_mode="sparse")
solver_options = {"method": "jax_odeint", "atol": 1e-6, "rtol": 1e-8}
pulse_backend.set_options(solver_options=solver_options)
pulse_backend.configuration = lambda: gate_backend.configuration()

In [ ]:
simplified_h2_molecule = """
H 0.0 0.0 0.0
H 0.0 0.0 0.735
"""

driver = PySCFDriver(
    atom=simplified_h2_molecule.strip(),
    basis='sto3g',
    charge=0,
    spin=0,
    unit=DistanceUnit.ANGSTROM
)

qmolecule = driver.run()

In [ ]:
from qiskit_nature import settings
settings.use_pauli_sum_op = False

In [ ]:
hamiltonian = qmolecule.hamiltonian
coefficients = hamiltonian.electronic_integrals
second_q_op = hamiltonian.second_q_op()
print(second_q_op)
nuclear_cont = hamiltonian.nuclear_repulsion_energy

In [ ]:
print(coefficients.alpha)

In [ ]:
print(coefficients.beta)

In [ ]:
print(coefficients.beta_alpha)

In [ ]:
import re

mapper = JordanWignerMapper()
qubit_op = mapper.map(second_q_op)
print(qubit_op)

my_dict = {}

for pauli in qubit_op:
    #print(pauli)
    str_info = pauli.__str__()
    print(f"Rappresentazione della stringa: {str_info}")

    # Estrai i coefficienti
    coeffs_start = str_info.find("coeffs=[") + len("coeffs=[")
    coeffs_end = str_info.find("]", coeffs_start)
    coeffs_str = str_info[coeffs_start:coeffs_end].strip()
    
    # Estrai gli operatori come stringa
    operators_start = str_info.find("Op(['") + len("Op(['")
    operators_end = str_info.find("'])", operators_start)
    operators_str = str_info[operators_start:operators_end]

    # Utilizza espressioni regolari per estrarre la parte reale e immaginaria dei coefficienti
    coeffs_match = re.findall(r"(-?\d+\.\d+)([+-]\d+\.\d+j)?", coeffs_str)

    # Inizializza i coefficienti come float
    real_coeffs = [float(match[0]) for match in coeffs_match]
    imag_coeffs = [float(match[1]) if match[1] else 0.0 for match in coeffs_match]

    # Somma i coefficienti complessi
    complex_coeff = sum(complex(real, imag) for real, imag in zip(real_coeffs, imag_coeffs))

    # Aggiungi il coefficiente all'operatore nel dizionario
    current_coeff = my_dict.get(operators_str, 0.0)
    my_dict[operators_str] = current_coeff + complex_coeff
    
print(my_dict)


In [ ]:
from qiskit.pulse import Schedule, GaussianSquare, Drag, Delay, Play, ControlChannel, DriveChannel

def drag_pulse(backend, amp, angle):
    backend_defaults = backend.defaults()
    inst_sched_map = backend_defaults.instruction_schedule_map
    x_pulse = inst_sched_map.get('x', (0)).filter(channels=[DriveChannel(0)], instruction_types=[Play]).instructions[0][1].pulse
    x_pulse = inst_sched_map.get('x', (0)).filter(channels=[DriveChannel(0)], instruction_types=[Play]).instructions[0][1].pulse
    duration_parameter = x_pulse.parameters['duration']
    sigma_parameter = x_pulse.parameters['sigma']
    beta_parameter = x_pulse.parameters['beta']
    pulse1 = Drag(duration=duration_parameter, sigma=sigma_parameter, beta=beta_parameter, amp=amp, angle=angle)
    return pulse1

In [ ]:
def cr_pulse(backend, amp, angle, duration):
    backend_defaults = backend.defaults()
    inst_sched_map = backend_defaults.instruction_schedule_map
    cr_pulse = inst_sched_map.get('cx', (0, 1)).filter(channels=[ControlChannel(0)], instruction_types=[Play]).instructions[0][1].pulse
    cr_params = {
        'duration': cr_pulse.parameters['duration'],
        'amp': cr_pulse.parameters['amp'],
        'angle': cr_pulse.parameters['angle'],
        'sigma': cr_pulse.parameters['sigma'],
        'width': cr_pulse.parameters['width']
    }
    cr_risefall = (cr_params['duration'] - cr_params['width']) / (2 * cr_params['sigma'])
    angle_parameter = angle
    duration_parameter = duration
    sigma_parameter = cr_pulse.parameters['sigma']
    width_parameter = int(duration_parameter - 2 * cr_risefall * cr_params['sigma'])
    pulse1 = GaussianSquare(duration=duration_parameter, amp=amp, angle=angle_parameter, sigma=sigma_parameter, width=width_parameter)

    return pulse1

In [ ]:
import qiskit.pulse as pulse
def HE_pulse(backend, amp, angle, width):
    # Crea un programma di impulsi vuoto
    with pulse.build(backend) as my_program1:
        sched_list = []
        # Layer 1: Applica una porta X su ciascun qubit
        with pulse.build(backend) as sched1:
            qubits = (0, 1, 2, 3, 4)
            for i in range(4):
                pulse.play(drag_pulse(backend, amp[i], angle[i]), DriveChannel(qubits[i]))
            sched_list.append(sched1)

        # Layer 2: Applica una porta di controllo tra qubit 0 e 1
        with pulse.build(backend) as sched2:
            uchan = pulse.control_channels(0, 1)[0]
            pulse.play(cr_pulse(backend, amp[4], angle[4], width[0]), uchan)
            sched_list.append(sched2)

        # Layer 3: Applica una porta di controllo tra qubit 1 e 2
        with pulse.build(backend) as sched4:
            uchan = pulse.control_channels(1, 2)[0]
            pulse.play(cr_pulse(backend, amp[5], angle[5], width[1]), uchan)
            sched_list.append(sched4)

        # Layer 4: Applica una porta di controllo tra qubit 2 e 3
        with pulse.build(backend) as sched6:
            uchan = pulse.control_channels(2, 3)[0]
            pulse.play(cr_pulse(backend, amp[6], angle[6], width[2]), uchan)
            sched_list.append(sched6)

        # Costruisci il programma principale allineando sequenzialmente gli schedule
        with pulse.build(backend) as my_program:
            with pulse.transpiler_settings(initial_layout=[0, 1, 2, 3]):
                with pulse.align_sequential():
                    for sched in sched_list:
                        pulse.call(sched)

    return my_program

In [ ]:
import copy
from scipy.optimize import minimize, LinearConstraint

def measurement_pauli(prepulse, pauli_string, backend, n_qubit):
    with pulse.build(backend) as pulse_measure:
        pulse.call(copy.deepcopy(prepulse))
        for ind, pauli in enumerate(pauli_string):
            if pauli == 'X':
                pulse.u2(0, np.pi, ind)
            elif pauli == 'Y':
                pulse.u2(0, np.pi / 2, ind)
        for qubit in range(n_qubit):
            pulse.barrier(qubit)
        pulse.measure(range(n_qubit))
    return pulse_measure

def n_one(bitstring, key):
    results = 0
    for ind, b in enumerate(reversed(bitstring)):
        if b == '1' and key[ind] != 'I':
            results += 1
    return results

def expectation_value(counts, shots, key):
    results = 0
    for bitstring in counts:
        if n_one(bitstring, key) % 2 == 1:
            results -= counts[bitstring] / shots
        else:
            results += counts[bitstring] / shots
    return results

def run_pulse_sim(meas_pulse, key, pulse_backend, backend, n_shot):
    results = pulse_backend.run(meas_pulse).result()
    counts = results.get_counts()
    expectation = expectation_value(counts, n_shot, key)
    return expectation

def gen_LC_vqe(parameters):
    lb = np.zeros(parameters)
    ub = np.ones(parameters)
    LC = LinearConstraint(np.eye(parameters), lb, ub, keep_feasible=False)
    return LC

In [ ]:
def vqe_one(prepulse, n_qubit, n_shot, pulse_backend, backend, key, value):
    all_Is = all(key_ele == 'I' for key_ele in key)
    if all_Is:
        return value

    meas_pulse = measurement_pauli(prepulse=prepulse, pauli_string=key, backend=backend, n_qubit=n_qubit)

    return value * run_pulse_sim(meas_pulse, key, pulse_backend, backend, n_shot)

In [ ]:
def vqe(params, pauli_dict, pulse_backend, backend, n_qubit, n_shot):
    print("params in def chemistry in vqe.py: ", params)

    width_len = int(len(params) - 1 * (n_qubit - 1))

    split_ind = int(width_len / 2)

    # Estrai le ampiezze, gli angoli e le larghezze dalla lista dei parametri
    amp = np.array(params[:split_ind])
    angle = np.array(params[split_ind:width_len]) * np.pi * 2
    width_1 = np.array(params[width_len:])

    # Calcola il numero di intervalli di larghezza normalizzati
    num_items = (1024 - 256) // 16 + 1

    # Normalizza le larghezze tra 256 e 1024
    width_norm = (width_1 - 256) / (1024 - 256)
    width_norm = np.clip(width_norm, 0, 1)

    # Calcola le larghezze in base alla normalizzazione
    width = (np.round(width_norm * (num_items - 1)) * 16 + 256).astype(int)

    amp = amp.tolist()
    angle = angle.tolist()
    width = width.tolist()

    keys = [key for key in pauli_dict]
    values = [pauli_dict[key] for key in pauli_dict]

    expect_values = []
    for key, value in zip(keys, values):
        # Genera l'impulso di preparazione
        prepulse = HE_pulse(backend, amp, angle, width)

        expect = vqe_one(prepulse, n_qubit, n_shot, pulse_backend, backend, key, value)
        expect_values.append(expect)

    print("E for cur_iter: ", sum(expect_values))

    return sum(expect_values)

In [ ]:
from qiskit.algorithms.optimizers import SPSA, SLSQP, COBYLA

iterazioni = []
energy = []

n_qubit = 4
parameters = 17
params = np.zeros(parameters)

LC = gen_LC_vqe(parameters)

n_shot = 1024

optimizer = 'COBYLA'

vqe_res = minimize(vqe, params, args=(my_dict, pulse_backend, gate_backend, n_qubit, n_shot),
                   method=optimizer, constraints=LC, options={'rhobeg': 0.2, 'maxiter': 1, 'disp': True})

iterazioni.append(1)
energy.append(vqe_res.fun)
print(my_dict)
print('The optimized loss func value: {}'.format(vqe_res.fun))

In [ ]:
from qiskit.algorithms.optimizers import SPSA, SLSQP, COBYLA

iterazioni = []
energy = []

n_qubit = 4
parameters = 17
params = np.zeros(parameters)

LC = gen_LC_vqe(parameters)

n_shot = 1024

optimizer = 'COBYLA'

vqe_res = minimize(vqe, params, args=(my_dict, pulse_backend, gate_backend, n_qubit, n_shot),
                   method=optimizer, constraints=LC, options={'rhobeg': 0.2, 'maxiter': 5, 'disp': True})

iterazioni.append(5)
energy.append(vqe_res.fun)
print(my_dict)
print('The optimized loss func value: {}'.format(vqe_res.fun))

In [ ]:
n_qubit = 4
parameters = 17
params = np.zeros(parameters)

LC = gen_LC_vqe(parameters)

n_shot = 1024

optimizer = 'COBYLA'

vqe_res = minimize(vqe, params, args=(my_dict, pulse_backend, gate_backend, n_qubit, n_shot),
                   method=optimizer, constraints=LC, options={'rhobeg': 0.2, 'maxiter': 10, 'disp': True})

iterazioni.append(10)
energy.append(vqe_res.fun)
print(my_dict)
print('The optimized loss func value: {}'.format(vqe_res.fun))

In [ ]:
n_qubit = 4
parameters = 17
params = np.zeros(parameters)

LC = gen_LC_vqe(parameters)

n_shot = 1024

optimizer = 'COBYLA'

vqe_res = minimize(vqe, params, args=(my_dict, pulse_backend, gate_backend, n_qubit, n_shot),
                   method=optimizer, constraints=LC, options={'rhobeg': 0.2, 'maxiter': 15, 'disp': True})

iterazioni.append(15)
energy.append(vqe_res.fun)
print(my_dict)
print('The optimized loss func value: {}'.format(vqe_res.fun))

In [ ]:
n_qubit = 4
parameters = 17
params = np.zeros(parameters)

LC = gen_LC_vqe(parameters)

n_shot = 1024

optimizer = 'COBYLA'

vqe_res = minimize(vqe, params, args=(my_dict, pulse_backend, gate_backend, n_qubit, n_shot),
                   method=optimizer, constraints=LC, options={'rhobeg': 0.2, 'maxiter': 20, 'disp': True})

iterazioni.append(20)
energy.append(vqe_res.fun)
print(my_dict)
print('The optimized loss func value: {}'.format(vqe_res.fun))

In [ ]:
import matplotlib.pyplot as plt
plt.plot(iterazioni, energy, label="PulseVQE")
plt.xlabel("Iterazioni")
plt.ylabel("Energy")
plt.legend()
plt.savefig('H2-pVQE.png')
plt.show()